In [1]:
import json
import time
import azureml
import logging
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Run, Datastore, Experiment

In [2]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')

ws = Workspace.from_config()

In [3]:
cluster = 'gandalf'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Found existing compute target


In [4]:
# Register datastore
#ds = Datastore.register_azure_blob_container(workspace=ws,
#                                             datastore_name='foodai',
#                                             container_name='foodai',
#                                             account_name='haldatasets',
#                                             account_key='<KEY>',
#                                             create_if_not_exists=False)

for name, ds in ws.datastores.items():
    print(name, ds.datastore_type)

foodai AzureBlob
absa AzureBlob
azureml_globaldatasets AzureBlob
surfrider AzureBlob
seer AzureBlob
smtprodwestus21globaluploadedresources AzureBlob
workspaceblobstore AzureBlob
workspacefilestore AzureFile


In [5]:
ds = Datastore.get(ws, datastore_name='foodai')
script_params = {
    "--data": ds.as_mount(), # don't download
    "--output": "outputs",
    "--epochs": 25,
    "--batch": 8,
    "--lr": 0.001,
}

# Create and run experiment
foodEstimator = PyTorch(source_directory='./',
                        script_params=script_params,
                        compute_target=compute,
                        entry_script='train.py',
                        conda_packages=['matplotlib'],
                        use_gpu=True)

WARNING - framework_version is not specified, defaulting to version 1.4.


In [6]:
exp = Experiment(ws, 'food_ai')
run = exp.submit(foodEstimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
food_ai,food_ai_1592855125_56b6acc8,azureml.scriptrun,Queued,Link to Azure Machine Learning studio,Link to Documentation
